In [15]:
import tensorflow as tf
from tensorflow import keras

In [16]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import numpy as np

In [18]:
fashion_mnist = keras.datasets.fashion_mnist

In [19]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [20]:
train_images = train_images/255.0
test_images = test_images/255.0

In [21]:
train_images[0].shape

(28, 28)

In [22]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)


In [23]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [24]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [25]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [26]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.8511666655540466

Best val_accuracy So Far: 0.8730000257492065
Total elapsed time: 00h 02m 48s


In [27]:
model=tuner_search.get_best_models(num_models=1)[0]

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        13856     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 48)                884784    
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                                 
Total params: 899,610
Trainable params: 899,610
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2928 - accuracy: 0.8899 - val_loss: 0.4306 - val_accuracy: 0.8685
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2713 - accuracy: 0.8977 - val_loss: 0.4129 - val_accuracy: 0.8797
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2476 - accuracy: 0.9063 - val_loss: 0.3893 - val_accuracy: 0.8800
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2341 - accuracy: 0.9135 - val_loss: 0.4069 - val_accuracy: 0.8768
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2252 - accuracy: 0.9161 - val_loss: 0.4257 - val_accuracy: 0.8653
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2029 - accuracy: 0.9245 - val_loss: 0.4695 - val_accuracy: 0.8735
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2049 - accuracy: 0.9248 - val_loss: 0.4787 - val_accuracy